In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re 


df = pd.read_csv('UNRATE.csv')
df['Year'] = df['observation_date'].str.extract(r'(\d{4})')
df['Month'] = df['observation_date'].str.extract(r'((?<=\d{4}-)\d{2})')
# df['Day'] = df['observation_date'].str.extract(r'((?<=\d{4}-\d{2}-)\d{2})')
df = df.drop('observation_date', axis = 1)
df['Month'] = df['Month'].astype(int)
df['Month'] = pd.to_datetime(df['Month'], format='%m').dt.month_name()
df['Year'] = df['Year'].astype(int)
column_order = ['Month', 'Year', 'UNRATE']
UNRATE = df[column_order]
UNRATE.head()

,Month,Year,UNRATE
0,January,1948,3.4
1,February,1948,3.8
2,March,1948,4.0
3,April,1948,3.9
4,May,1948,3.5


In [2]:
df= pd.read_csv('GDP.csv')
df = df.drop([0,1,2,3])
df['Year'] = df['observation_date'].str.extract(r'(\d{4})')
df['Month'] = df['observation_date'].str.extract(r'((?<=\d{4}-)\d{2})')
df = df.drop('observation_date', axis = 1)
df['Month'] = df['Month'].astype(int)
df['Month'] = pd.to_datetime(df['Month'], format='%m').dt.month_name()
df['Year'] = df['Year'].astype(int)
column_order = ['Month', 'Year', 'GDP']
GDP = df[column_order]
GDP.head()

,Month,Year,GDP
4,January,1948,265.742
5,April,1948,272.567
6,July,1948,279.196
7,October,1948,280.366
8,January,1949,275.034


In [3]:
df = pd.read_csv('CPI.csv')
df = df.drop(df.index[0:12])
df['Year'] = df['observation_date'].str.extract(r'(\d{4})')
df['Month'] = df['observation_date'].str.extract(r'((?<=\d{4}-)\d{2})')
df['CPI'] = df['CPIAUCSL']
df = df.drop('observation_date', axis = 1)
df = df.drop('CPIAUCSL', axis = 1)
df['Month'] = df['Month'].astype(int)
df['Month'] = pd.to_datetime(df['Month'], format='%m').dt.month_name()
df['Year'] = df['Year'].astype(int)
column_order = ['Month', 'Year', 'CPI']
CPI = df[column_order]
CPI.head()

,Month,Year,CPI
12,January,1948,23.68
13,February,1948,23.67
14,March,1948,23.50
15,April,1948,23.82
16,May,1948,24.01


In [4]:
combined = pd.merge(UNRATE, GDP, on=['Month', 'Year'], how='outer')
combined.head()
data = pd.merge(combined, CPI, on=['Year', 'Month'], how='outer') 
month_order = [
    "January", "February", "March", "April", "May", "June",
    "July", "August", "September", "October", "November", "December"
]
data["Month"] = pd.Categorical(data["Month"], categories=month_order, ordered=True)
data = data.sort_values(["Year", "Month"]).reset_index(drop=True)
data.head(20)



,Month,Year,UNRATE,GDP,CPI
0,January,1948,3.4,265.742,23.68
1,February,1948,3.8,NaN,23.67
2,March,1948,4.0,NaN,23.50
3,April,1948,3.9,272.567,23.82
4,May,1948,3.5,NaN,24.01
5,June,1948,3.6,NaN,24.15
6,July,1948,3.6,279.196,24.40
7,August,1948,3.9,NaN,24.43
8,September,1948,3.8,NaN,24.36
9,October,1948,3.7,280.366,24.31


In [5]:
import requests 
import re 
from bs4 import BeautifulSoup 
import pandas as pd 
import numpy as np       
url = 'https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States' 
email = "skirk03@byu.edu" 
ua = f"STAT386-class-scraper/1.0 (+{email})" 
r = requests.get(url, headers={"User-Agent": ua, "From": email}, timeout=15)


In [6]:
bs = BeautifulSoup(r.text, 'html.parser')
tables = pd.read_html(r.text)
df = tables[0]
df.columns
df["No."] = df["No.[a]"]
df['President'] = df['Name (birth–death)'].str.extract(r'([A-Z][a-z]+\W[A-Z][a-z]+)')
df['Party'] = df['Party[b][17].1']
df[['Start', 'End']] = df['Term[16]'].str.split('–', expand=True)
df['Start_Month'] = df['Start'].str.extract(r'([A-Z][a-z]+)')
df['Start_Year'] = df['Start'].str.extract(r'(\d{4})')
df['End_Month'] = df['End'].str.extract(r'([A-Z][a-z]+)')
df['End_Year'] = df['End'].str.extract(r'(\d{4})')


df = df.drop('Portrait', axis = 1)
df = df.drop('Party[b][17]', axis = 1)
df = df.drop('Vice President[18]', axis = 1)
df = df.drop('No.[a]', axis = 1)
df = df.drop('Name (birth–death)', axis = 1)
df = df.drop('Party[b][17].1', axis = 1)
df = df.drop('Term[16]', axis = 1)
df = df.drop('Start', axis = 1)
df = df.drop('End', axis = 1)


/var/folders/g0/nsxm0s0x64z69dg0l2b9mf4m0000gn/T/ipykernel_73166/1178456700.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(r.text)


In [7]:
df['End_Year'] = df['End_Year'].fillna(2025)
df.loc[df['End_Month'] == 'Incumbent', 'End_Month'] = 'November'
df.tail()

df['Start_Date'] = pd.to_datetime(df['Start_Month'] + " " + df['Start_Year'].astype(str))
df['End_Date'] = pd.to_datetime(df['End_Month'] + " " + df['End_Year'].astype(str))

rows = []

for _, row in df.iterrows():
    dates = pd.date_range(start=row['Start_Date'], end=row['End_Date'], freq='MS')
    
    for d in dates:
        new_row = row.to_dict()          # keep ALL original columns
        new_row['Year'] = d.year
        new_row['Month'] = d.month_name()
        new_row['Month_Num'] = d.month
        rows.append(new_row)

Presidents = pd.DataFrame(rows)
Presidents = Presidents.drop('Month_Num', axis = 1)
Presidents = Presidents.drop('Start_Month', axis = 1)
Presidents = Presidents.drop('Start_Year', axis = 1)
Presidents = Presidents.drop('End_Month', axis = 1)
Presidents = Presidents.drop('End_Year', axis = 1)
Presidents = Presidents.drop('Start_Date', axis = 1)
Presidents = Presidents.drop('End_Date', axis = 1)
Presidents = Presidents.drop('Election', axis = 1)
Presidents = Presidents.drop('No.', axis = 1)
Presidents = Presidents[Presidents['Year'] >= 1948]
column_order = ['Month', 'Year', 'President', 'Party']
Presidents = Presidents[column_order]

/var/folders/g0/nsxm0s0x64z69dg0l2b9mf4m0000gn/T/ipykernel_73166/1740070359.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Start_Date'] = pd.to_datetime(df['Start_Month'] + " " + df['Start_Year'].astype(str))
/var/folders/g0/nsxm0s0x64z69dg0l2b9mf4m0000gn/T/ipykernel_73166/1740070359.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['End_Date'] = pd.to_datetime(df['End_Month'] + " " + df['End_Year'].astype(str))


In [8]:
Final = pd.merge(data, Presidents, on=['Year', 'Month'], how='outer')
Final.head(500)

,Month,Year,UNRATE,GDP,CPI,President,Party
0,April,1948,3.9,272.567,23.82,NaN,Democratic
1,April,1948,3.9,272.567,23.82,NaN,Democratic
2,August,1948,3.9,NaN,24.43,NaN,Democratic
3,August,1948,3.9,NaN,24.43,NaN,Democratic
4,December,1948,4.0,NaN,24.05,NaN,Democratic
...,...,...,...,...,...,...,...
495,January,1969,3.4,993.337,35.70,Richard Nixon,Republican
496,January,1969,3.4,993.337,35.70,Richard Nixon,Republican
497,July,1969,3.5,1029.956,36.80,Richard Nixon,Republican
498,July,1969,3.5,1029.956,36.80,Richard Nixon,Republican


In [9]:
Final["Month"] = Final["Month"].str.strip()
Final["President"] = Final["President"].str.strip()


month_map = {
    "January": 1, "February": 2, "March": 3, "April": 4,
    "May": 5, "June": 6, "July": 7, "August": 8,
    "September": 9, "October": 10, "November": 11, "December": 12,
}


terms = [
    ("Harry S Truman",  (1945, 'April'),  (1953, 'January')),
    ("Dwight D Eisenhower", (1953, 'January'), (1961, 'January')),
    ("John F Kennedy",  (1961, 'January'), (1963, 'November')),
    ("Lyndon B Johnson", (1963, 'November'), (1969, 'January')),
]

for i, row in Final.iterrows():
    if pd.isna(row["President"]):

        current = (row["Year"], month_map[row["Month"]])

        for pres, (sy, sm), (ey, em) in terms:
            start = (sy, month_map[sm])
            end   = (ey, month_map[em])

            if start <= current < end:
                Final.loc[i, "President"] = pres
                break

Final.iloc[100:150]



,Month,Year,UNRATE,GDP,CPI,President,Party
100,December,1952,2.7,NaN,26.71,Harry S Truman,Democratic
101,December,1952,2.7,NaN,26.71,Harry S Truman,Democratic
102,February,1952,3.1,NaN,26.41,Harry S Truman,Democratic
103,February,1952,3.1,NaN,26.41,Harry S Truman,Democratic
104,January,1952,3.2,359.820,26.45,Harry S Truman,Democratic
105,January,1952,3.2,359.820,26.45,Harry S Truman,Democratic
106,July,1952,3.2,367.701,26.68,Harry S Truman,Democratic
107,July,1952,3.2,367.701,26.68,Harry S Truman,Democratic
108,June,1952,3.0,NaN,26.53,Harry S Truman,Democratic
109,June,1952,3.0,NaN,26.53,Harry S Truman,Democratic


In [10]:
month_order = [
    "January", "February", "March", "April", "May", "June",
    "July", "August", "September", "October", "November", "December"
]
Final["Month"] = pd.Categorical(Final["Month"], categories=month_order, ordered=True)
Final = Final.sort_values(["Year", "Month"]).reset_index(drop=True)
Final.head(20)

,Month,Year,UNRATE,GDP,CPI,President,Party
0,January,1948,3.4,265.742,23.68,Harry S Truman,Democratic
1,January,1948,3.4,265.742,23.68,Harry S Truman,Democratic
2,February,1948,3.8,NaN,23.67,Harry S Truman,Democratic
3,February,1948,3.8,NaN,23.67,Harry S Truman,Democratic
4,March,1948,4.0,NaN,23.50,Harry S Truman,Democratic
5,March,1948,4.0,NaN,23.50,Harry S Truman,Democratic
6,April,1948,3.9,272.567,23.82,Harry S Truman,Democratic
7,April,1948,3.9,272.567,23.82,Harry S Truman,Democratic
8,May,1948,3.5,NaN,24.01,Harry S Truman,Democratic
9,May,1948,3.5,NaN,24.01,Harry S Truman,Democratic


In [11]:
Final = Final.drop_duplicates()
Final.tail()


,Month,Year,UNRATE,GDP,CPI,President,Party
1787,July,2025,4.2,NaN,322.132,Donald Trump,Republican
1788,August,2025,4.3,NaN,323.364,Donald Trump,Republican
1789,September,2025,4.4,NaN,324.368,Donald Trump,Republican
1790,October,2025,NaN,NaN,NaN,Donald Trump,Republican
1791,November,2025,NaN,NaN,NaN,Donald Trump,Republican


In [12]:
# Filling in the missing data with the mean of the previous and next known values
Final["GDP"] = Final["GDP"].interpolate(method="linear") \
                           .ffill()
Final = Final.iloc[:-2]
Final.head(100)


,Month,Year,UNRATE,GDP,CPI,President,Party
0,January,1948,3.4,265.742000,23.68,Harry S Truman,Democratic
2,February,1948,3.8,268.017000,23.67,Harry S Truman,Democratic
4,March,1948,4.0,270.292000,23.50,Harry S Truman,Democratic
6,April,1948,3.9,272.567000,23.82,Harry S Truman,Democratic
8,May,1948,3.5,274.776667,24.01,Harry S Truman,Democratic
...,...,...,...,...,...,...,...
190,November,1955,4.2,437.976667,26.88,Dwight D Eisenhower,Republican
192,December,1955,4.2,438.861333,26.87,Dwight D Eisenhower,Republican
194,January,1956,4.0,439.746000,26.83,Dwight D Eisenhower,Republican
196,February,1956,3.9,441.834000,26.86,Dwight D Eisenhower,Republican


In [13]:
Final.to_csv("econ_data.csv")